### We take raw data from 'data/' folder , populate indicators and write it to 'data_with_indicators' folder

In [1]:
import pandas as pd
import os
import pandas_ta as ta
import yfinance as yf
import datetime

### This part is for updating raw OHLCV files from 'data/' folder.
#### TODO: Consider to remove this part as OHLCV update is done in another script: update_history_ohlcv.ipynb

In [13]:
# Initialize an empty list for symbols
symbols = []

# Specify the directory path
data_folder = 'data/'
indicators_data_folder = 'data_with_indicators/'

# Iterate over each file in the directory
# We assume there are the same files names in two folders: 'data/' and 'data_with_indicators/'
for filename in os.listdir(data_folder):
    # Check if the file ends with .csv
    if filename.endswith('.csv'):
        # Remove the .csv extension and add the symbol to the list
        symbol = filename.split('.')[0]
        symbols.append(symbol)

symbols.sort()

In [ ]:
def update_ohlcv_data(symbols, data_folder):
    for symbol in symbols:
        # Set the file path
        raw_data_file = os.path.join(data_folder, f"{symbol}.csv")
        
        # Check if OHLCV file exists
        if os.path.exists(raw_data_file):
            # Read existing data
            existing_data = pd.read_csv(raw_data_file, parse_dates=['Date'])
            
            # Get the last date in the existing data
            last_date = existing_data['Date'].max()
            
            # Fetch new data from the last date till today -
            # we are updating last day as well, because Volume may change after market close
            start_date = last_date
            new_data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), interval='1d')
            
            # If new data exists, append it to the existing data
            if not new_data.empty:
                new_data.reset_index(inplace=True)  # Reset index to make 'Date' a column
                new_data['Date'] = pd.to_datetime(new_data['Date'])  # Ensure date format consistency

                # Remove the last record from existing_data, because Volume is updated even till the next day
                existing_data = existing_data.iloc[:-1]
                # Concatenate existing_data with new_data
                updated_data = pd.concat([existing_data, new_data], ignore_index=True)
                
                # Save the updated data back to the CSV file
                updated_data.to_csv(raw_data_file, index=False)

                print(f"{symbol}: OHLCV updated successfully.")
            else:
                print(f"{symbol}: No new data to update.")
        else:
            print(f"{symbol}: OHLCV file not found.")

In [ ]:
# Run the function
update_ohlcv_data(symbols, data_folder)

### This part is for updating files with indicators

In [19]:
# Initialize an empty list for symbols
symbols = []

# Specify the directory path
indicators_data_folder = 'data_with_indicators/'

# Iterate over each file in the directory
for filename in os.listdir(indicators_data_folder):
    # Check if the file ends with .csv
    if filename.endswith('.csv'):
        # Remove the .csv extension and add the symbol to the list
        symbol = filename.split('.')[0]
        symbols.append(symbol)

symbols.sort()

In [23]:
def update_indicators_data(symbols, indicators_data_folder):
    for symbol in symbols:
        # Set the file path
        indicators_data_file = os.path.join(indicators_data_folder, f"{symbol}.csv")
        
        # Check if OHLCV file exists
        if os.path.exists(indicators_data_file):
            # Read existing data
            existing_data = pd.read_csv(indicators_data_file, parse_dates=['Date'])
            
            # Get the last date in the existing data
            last_date = existing_data['Date'].max()
            
            # Fetch new data from the last date till today -
            # we are updating last day as well, because Volume may change after market close
            start_date = last_date
            new_data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), interval='1d')
            
            # If new data exists, append it to the existing data
            if not new_data.empty:
                new_data.reset_index(inplace=True)  # Reset index to make 'Date' a column
                new_data['Date'] = pd.to_datetime(new_data['Date'])  # Ensure date format consistency
                # print("New data: \n", new_data)
                # print("Existing dataa: ", existing_data)
                # Remove the last record from existing_data, because Volume is updated even till the next day
                existing_data = existing_data.iloc[:-1]
                # print("Existing after removing last: ", existing_data)
                
                # Concatenate existing_data with new_data
                updated_data = pd.concat([existing_data, new_data], ignore_index=True)
                
                # UPDATING INDICATORS
                #updated_data.loc[updated_data['Date'] >= start_date,'SMA_10'] = updated_data['Close'].rolling(window=10).mean()  # n-day Moving Average

                # SMA:
                sma_windows = [10, 20, 50, 100, 200]
                for sma_window in sma_windows:
                    updated_data.loc[updated_data['Date'] >= start_date,f'SMA_{sma_window}'] = updated_data['Close'].rolling(window=sma_window).mean()  # n-day Moving Average
    
                # Bollinger Bands - for 20, 50, 200 windows
                bb_windows = [20, 50, 200]
                bb_deviation = 2.0
                for bb_window in bb_windows:
                    updated_data.loc[updated_data['Date'] >= start_date,f'BB_{bb_window}_upper'] = updated_data[f'SMA_{bb_window}'] + (updated_data['Close'].rolling(window=bb_window).std() * bb_deviation)  # Upper Band
                    updated_data.loc[updated_data['Date'] >= start_date,f'BB_{bb_window}_lower'] = updated_data[f'SMA_{bb_window}'] - (updated_data['Close'].rolling(window=bb_window).std() * bb_deviation)  # Lower Band
    
                # RSI:
                rsi_windows = [7, 14, 30]
                for rsi_window in rsi_windows:
                    updated_data.loc[updated_data['Date'] >= start_date,f'RSI_{rsi_window}'] = ta.rsi(updated_data['Close'], length=rsi_window)
    
                # Avg_Volume
                avg_vol_windows = [20, 40, 80]
                for vol_window in avg_vol_windows:
                    updated_data.loc[updated_data['Date'] >= start_date,f'Avg_Volume_{vol_window}'] = updated_data['Volume'].rolling(window=vol_window).mean()

                # Optimize data types
                for col in updated_data.columns:
                    if updated_data[col].dtype == 'float64':
                        updated_data[col] = updated_data[col].astype('float32')  # Convert to float32
                
                # print("TEST: \n", updated_data.tail(7))

                # print("Updated data (last 7 rows): \n", updated_data.tail(7))
                # return
                # Save the updated data back to the CSV file
                updated_data.to_csv(indicators_data_file, index=False)

                print(f"{symbol}: Indicators file updated successfully.")
            else:
                print(f"{symbol}: No new data to update.")
        else:
            print(f"{symbol}: Indicators data file not found.")




In [25]:
update_indicators_data(symbols, indicators_data_folder)

[*********************100%%**********************]  1 of 1 completed

/home/serguz/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



AAL: Indicators file updated successfully.
[*********************100%%**********************]  1 of 1 completed

/home/serguz/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



AAME: Indicators file updated successfully.
[*********************100%%**********************]  1 of 1 completed

...

/home/serguz/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



ZUMZ: Indicators file updated successfully.
[*********************100%%**********************]  1 of 1 completed

/home/serguz/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



ZVRA: Indicators file updated successfully.
[*********************100%%**********************]  1 of 1 completed
ZYXI: Indicators file updated successfully.


/home/serguz/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
